# Loading libraries and data

In [5]:
import pandas as pd
from datetime import datetime

In [6]:
df_train = pd.read_csv('data/event-log-training.csv')

In [7]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000
...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000


# Convert timestamp to datetime

In [8]:
date_list = []

for time in df_train['event time:timestamp']:
    datex = time[:-4]
    date = datetime.strptime(datex, '%d-%m-%Y %H:%M:%S')

    date_list.append(date)

In [9]:
df_train['time and date'] = date_list

In [10]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00
...,...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,1970-12-25 15:51:00
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000,1970-12-25 16:05:00
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,1970-12-25 16:10:00
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,1970-12-25 16:22:00


# Add in true next event and time it takes for case 4

In [11]:
df_train['event lifecycle:transition'].value_counts()

complete    33802
start        2826
Name: event lifecycle:transition, dtype: int64

In [12]:
dct_events = dict(df_train['event concept:name'].value_counts())

In [13]:
df_train['case concept:name'].value_counts()

12    2140
33    1594
42    1390
24    1285
6     1198
      ... 
27      62
80      59
56      55
4       43
51      43
Name: case concept:name, Length: 80, dtype: int64

In [14]:
df_train.groupby(by = ['case concept:name', 'time and date'], as_index = False)
df_case4 = df_train[df_train['case concept:name'] == 4]

In [15]:
#df_case4

Add event column

In [16]:
event_lst4 = [event for event in df_case4['event concept:name']]

event_lst4 = event_lst4[1:]
event_lst4.append('-')
    

In [17]:
df_case4['next_event'] = event_lst4

C:\Users\20204502\AppData\Local\Temp/ipykernel_22560/3880512715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['next_event'] = event_lst4


Add time to next event column

In [18]:
nexttime_lst1 = [time for time in df_case4['time and date']]

nexttime_lst = nexttime_lst1[1:]
nexttime_lst.append(nexttime_lst[-1])

In [19]:
time_diff = []

for i in range(len(nexttime_lst)):
    time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [20]:
df_case4['time to next event'] = time_diff

C:\Users\20204502\AppData\Local\Temp/ipykernel_22560/1624469924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['time to next event'] = time_diff


In [1]:
df_case4.head()

NameError: name 'df_case4' is not defined

# Next event and time column for all data


In [29]:
cases = list(df_train['case concept:name'].unique())

In [19]:
df_result = pd.DataFrame()

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00
...,...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,1970-12-25 15:51:00
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000,1970-12-25 16:05:00
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,1970-12-25 16:10:00
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,1970-12-25 16:22:00


In [ ]:
#create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [105]:
#2140
position_dict = {}

for i in range(2140):
    init = 0
    task = 0
    for key in int_dict.keys():
        try:
            new = int_dict[key][i]
        except:
            new = 0
        if new > init:
            init = new
            task = key

    position_dict.update({i: task})

In [110]:
pred_list = [position_dict[i] for i in position_dict.keys()]


In [111]:
pred_list

['Job',
 'Remote Print',
 'Read Print Options',
 'Rasterization',
 'Interpretation',
 'Move Scan Head',
 'Interpretation',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Accumulate Images',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Collect Image',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Illuminate Document',
 'Collect Image',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filtered Image',
 'Collect Image',
 'Illuminate Document',
 'Move Scan Head',
 'Focus Light Beam',
 'A/D Conversion',
 'Interpolation',
 'Filter

In [120]:
df_result = pd.DataFrame()

for case in cases: 
    
    df_case = df_train[df_train['case concept:name'] == case].copy().reset_index()

    pred_list = [position_dict[i] for i in range(len(df_case))]
    pred_list = pred_list[1:]
    pred_list.append('-')

    

    
    #create next event list
    event_lst = [event for event in df_case['event concept:name']]
    event_lst = event_lst[1:]
    event_lst.append('-')
    
    #create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])
    
    #append columns to df case
    df_case['Next event'] = event_lst
    df_case['Time to next event'] = time_diff


    df_case['Event predcition'] = pred_list
    
    #append df case to final df
    df_result = df_result.append(df_case)

In [ ]:
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)

In [121]:
df_result

,index,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date,Next event,Time to next event,Event predcition
0,0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00,Remote Print,0 days 00:15:00,Remote Print
1,1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00,Read Print Options,0 days 00:11:00,Read Print Options
2,2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00,Rasterization,0 days 00:12:00,Rasterization
3,3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00,Interpretation,0 days 00:13:00,Interpretation
4,4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00,Unformatted Text,0 days 00:18:00,Move Scan Head
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,9579,8589934665,100,Simulated process instance,Copy/Scan,PRN1,Compression,complete,29-03-1970 16:25:00.000,1970-03-29 16:25:00,Store Image,0 days 00:24:00,Screening
74,9580,8589934666,100,Simulated process instance,Copy/Scan,PRN1,Store Image,complete,29-03-1970 16:49:00.000,1970-03-29 16:49:00,Transfer Image,0 days 00:17:00,Move Scan Head
75,9581,8589934667,100,Simulated process instance,Copy/Scan,PRN1,Transfer Image,complete,29-03-1970 17:06:00.000,1970-03-29 17:06:00,Send SMTP,0 days 00:16:00,Screening
76,9582,8589934668,100,Simulated process instance,Copy/Scan,PRN1,Send SMTP,complete,29-03-1970 17:22:00.000,1970-03-29 17:22:00,Job,0 days 00:11:00,A/D Conversion


In [106]:
position_dict

{0: 'Job',
 1: 'Remote Print',
 2: 'Read Print Options',
 3: 'Rasterization',
 4: 'Interpretation',
 5: 'Move Scan Head',
 6: 'Interpretation',
 7: 'A/D Conversion',
 8: 'Interpolation',
 9: 'Filtered Image',
 10: 'Collect Image',
 11: 'Illuminate Document',
 12: 'Move Scan Head',
 13: 'Focus Light Beam',
 14: 'A/D Conversion',
 15: 'Interpolation',
 16: 'Filtered Image',
 17: 'Illuminate Document',
 18: 'Accumulate Images',
 19: 'Move Scan Head',
 20: 'Focus Light Beam',
 21: 'A/D Conversion',
 22: 'Interpolation',
 23: 'Filtered Image',
 24: 'Collect Image',
 25: 'Illuminate Document',
 26: 'Move Scan Head',
 27: 'Focus Light Beam',
 28: 'A/D Conversion',
 29: 'Interpolation',
 30: 'Filtered Image',
 31: 'Illuminate Document',
 32: 'Collect Image',
 33: 'Move Scan Head',
 34: 'Focus Light Beam',
 35: 'A/D Conversion',
 36: 'Interpolation',
 37: 'Filtered Image',
 38: 'Illuminate Document',
 39: 'Collect Image',
 40: 'Move Scan Head',
 41: 'Focus Light Beam',
 42: 'A/D Conversion',
 4

In [59]:
int_dict['Remote Print']

{1: 44}

In [42]:
dict = {1 : {2: 4}}

In [44]:
dict[1].update({5: 6})

In [45]:
dict

{1: {2: 4, 5: 6}}

Prediction stuff

In [22]:
lst = df_result[['event concept:name', 'Next event']].apply(tuple, axis=1)

In [23]:
lst = [elm for elm in lst]

In [24]:
count_dict = {}
for elm in lst:
    if elm in count_dict:
        count_dict[elm] +=1
    else:
        count_dict[elm] = 1

In [25]:
count_dict

{('Job', 'Remote Print'): 44,
 ('Remote Print', 'Read Print Options'): 44,
 ('Read Print Options', 'Rasterization'): 44,
 ('Rasterization', 'Interpretation'): 44,
 ('Interpretation', 'Unformatted Text'): 34,
 ('Unformatted Text', 'Interpretation'): 32,
 ('Interpretation', 'Rendering'): 75,
 ('Rendering', 'Screening'): 145,
 ('Screening', 'Interpretation'): 249,
 ('Interpretation', 'FM Screening'): 81,
 ('FM Screening', 'Unformatted Text'): 5,
 ('Unformatted Text', 'Screening'): 27,
 ('Interpretation', 'Current Page Image'): 111,
 ('Current Page Image', 'Interpretation'): 110,
 ('Interpretation', 'Accumulate Images'): 84,
 ('Accumulate Images', 'Rendering'): 192,
 ('Rendering', 'Unformatted Text'): 13,
 ('Interpretation', 'AM Screening'): 51,
 ('AM Screening', 'Interpretation'): 57,
 ('Screening', 'Current Page Image'): 249,
 ('Accumulate Images', 'Interpretation'): 40,
 ('Interpretation', 'Screening'): 201,
 ('Screening', 'Unformatted Text'): 21,
 ('Unformatted Text', 'Current Page Ima

In [ ]:
df_train[df_train['eventID '] == 146028888106]

In [ ]:
df_train.head()